# Open3D Guide: 11. Voxelization

Source: 

- Tutorial: [https://www.open3d.org/docs/latest/tutorial/Advanced/voxelization.html](https://www.open3d.org/docs/latest/tutorial/Advanced/voxelization.html).
- [`open3d.geometry.VoxelGrid`](https://www.open3d.org/docs/latest/python_api/open3d.geometry.VoxelGrid.html#open3d.geometry.VoxelGrid).
- [`open3d.geometry.Voxel`](https://www.open3d.org/docs/latest/python_api/open3d.geometry.Voxel.html#open3d.geometry.Voxel)

Summary of contents:

- A
- B

In [1]:
import sys
import os
import copy

# Add the directory containing 'examples' to the Python path
notebook_directory = os.getcwd()
parent_directory = os.path.dirname(notebook_directory)  # Parent directory
sys.path.append(parent_directory)

In [2]:
import open3d as o3d
import open3d.core as o3c
from examples import open3d_example as o3dex
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Voxelize from triangle mesh

In [3]:
# Get mesh
print('Input mesh')
mesh = o3dex.get_bunny_mesh()

# Optional: Fit to unit cube
#mesh.scale(
#    1.0 / np.max(mesh.get_max_bound() - mesh.get_min_bound()),
#    center=mesh.get_center()
#)

# Optional, for nice visualization
mesh.compute_vertex_normals()

# Visualize
o3d.visualization.draw_geometries([mesh])

Input mesh


In [4]:
# See all the properties of VoxelGrid
# https://www.open3d.org/docs/latest/python_api/open3d.geometry.VoxelGrid.html#open3d.geometry.VoxelGrid
print('Voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(
    mesh,
    voxel_size=0.001
)
o3d.visualization.draw_geometries([voxel_grid])

Voxelization


## Voxels

In [5]:
# Get OCCUPIED voxels: get_voxels()
# Voxel: https://www.open3d.org/docs/latest/python_api/open3d.geometry.Voxel.html#open3d.geometry.Voxel
# Voxel properties: grid_index (3,), color (3,)
# NOTE: Voxels are not thought for setting values manually...
for i, voxel in enumerate(voxel_grid.get_voxels()):
    print("grid_index: ", voxel.grid_index) # an array if 3 ints
    print("color: ", voxel.color) # an array of  floats; we can modify colors to display better!
    if i > 10:
        break

grid_index:  [  4 106  80]
color:  [0. 0. 0.]
grid_index:  [110  17 107]
color:  [0. 0. 0.]
grid_index:  [72 93 80]
color:  [0. 0. 0.]
grid_index:  [ 64  77 100]
color:  [0. 0. 0.]
grid_index:  [129  41 101]
color:  [0. 0. 0.]
grid_index:  [  8 115  70]
color:  [0. 0. 0.]
grid_index:  [ 97  52 119]
color:  [0. 0. 0.]
grid_index:  [ 29 113  40]
color:  [0. 0. 0.]
grid_index:  [146  33  62]
color:  [0. 0. 0.]
grid_index:  [ 18 123  74]
color:  [0. 0. 0.]
grid_index:  [30  1 72]
color:  [0. 0. 0.]
grid_index:  [ 16 115 102]
color:  [0. 0. 0.]


## Voxel cubes for visualization

In [6]:
# Get the voxel size and origin for computation
voxel_size = voxel_grid.voxel_size
origin = voxel_grid.origin

# Prepare to merge meshes
mesh = o3d.geometry.TriangleMesh()
for voxel in voxel_grid.get_voxels():
    # Calculate the center of the voxel
    voxel_center = origin + voxel.grid_index * voxel_size + voxel_size / 2.0
    # Create a cube mesh for each voxel
    cube = o3d.geometry.TriangleMesh.create_box(width=voxel_grid.voxel_size,
                                                height=voxel_grid.voxel_size,
                                                depth=voxel_grid.voxel_size)
    cube.translate(voxel_center - np.array([voxel_grid.voxel_size / 2] * 3))
    cube.paint_uniform_color([voxel_center[0] % 1, voxel_center[1] % 1, voxel_center[2] % 1])  # Set color
    mesh += cube

# Visualize the merged mesh
o3d.visualization.draw_geometries([mesh])

## Create a Voxelmap from the VoxelGrid: A cartesian occupancy map

In [12]:
# Get the extent of the voxel grid and compute dimensions
min_bound = voxel_grid.get_min_bound()
max_bound = voxel_grid.get_max_bound()
voxel_size = voxel_grid.voxel_size
aabb = voxel_grid.get_axis_aligned_bounding_box()
print(f"voxel_grid.min_bound: {min_bound}")
print(f"voxel_grid.max_bound: {max_bound}")
print(f"voxel_grid.voxel_size: {voxel_size}")
print(f"AABB: {aabb}")

# Compute grid dimensions
Nx = int((max_bound[0] - min_bound[0]) / voxel_size)
Ny = int((max_bound[1] - min_bound[1]) / voxel_size)
Nz = int((max_bound[2] - min_bound[2]) / voxel_size)
print(f"Number of occupied voxels: {len(voxel_grid.get_voxels())}")
print(f"Number of total voxels: {Nx*Ny*Nz}")

# Create an empty numpy array for the voxel map
voxel_map = np.zeros((Nx, Ny, Nz), dtype=bool)

# Populate the voxel map
solid_voxel_count = 0
for voxel in voxel_grid.get_voxels():
    # NOTE: a voxel_index is a a tuple (3,) with ints
    # and we get a voxel_index for each occupied voxel!
    index = voxel.grid_index
    if 0 <= index[0] < Nx and 0 <= index[1] < Ny and 0 <= index[2] < Nz:
        voxel_map[index] = True
        solid_voxel_count += 1

# Output the shape of the voxel map
print("Voxel map shape:", voxel_map.shape)
print("Found solid voxels: ", solid_voxel_count)

voxel_grid.min_bound: [-0.0951899  0.0324874 -0.0623736]
voxel_grid.max_bound: [0.0618101 0.1874874 0.0596264]
voxel_grid.voxel_size: 0.001
AABB: AxisAlignedBoundingBox: min: (-0.0951899, 0.0324874, -0.0623736), max: (0.0618101, 0.187487, 0.0596264)
Number of occupied voxels: 82999
Number of total voxels: 2968870
Voxel map shape: (157, 155, 122)
Found solid voxels:  82999


In [18]:
# Now, we check that voxel_map is correct by plotting the voxel centers as points

# Create a point cloud
pcd = o3d.geometry.PointCloud()

# List to hold points and colors
points = []
colors = []

# Iterate through the voxel map
# Faster alternative, but without using voxel_map
# for voxel in voxel_grid.get_voxels():
#     point = voxel_grid.get_voxel_center_coordinate(voxel.grid_index)
#     points.append(point)
#     colors.append([1, 0, 0])  # Red color for occupied voxels
for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            if voxel_map[i, j, k]:  # Check if the voxel is occupied
                # Compute the center of the voxel
                # The manual computation with min_bound is WRONG?!
                # center_x_ = min_bound[0] + (i * voxel_size) + (voxel_size*0.5)
                # center_y_ = min_bound[1] + (j * voxel_size) + (voxel_size*0.5)
                # center_z_ = min_bound[2] + (k * voxel_size) + (voxel_size*0.5)
                (center_x, center_y, center_z) = voxel_grid.get_voxel_center_coordinate(np.asarray([i,j,k]))
                points.append([center_x, center_y, center_z])
                colors.append([1, 0, 0])
                
# Convert lists to Open3D vectors
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

## Voxelize from a point cloud

In [3]:
# The voxel grid can also be created from a point cloud using the method create_from_point_cloud. 
# A voxel is occupied if at least one point of the point cloud is within the voxel. 
# The color of the voxel is the average of all the points within the voxel. 
# The argument voxel_size defines the resolution of the voxel grid.
print('Input')
N = 2000
pcd = o3dex.get_armadillo_mesh().sample_points_poisson_disk(N)
# Optional: Fit to unit cube
pcd.scale(1.0 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
o3d.visualization.draw_geometries([pcd])

print('voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.05)
o3d.visualization.draw_geometries([voxel_grid])

Input
voxelization


## Inclusion test

In [5]:
# The voxel grid can also be used to test if points are within an occupied voxel.
# The method check_if_included takes a (n,3) array as input and outputs a bool array.
queries = np.asarray(pcd.points)
output = voxel_grid.check_if_included(o3d.utility.Vector3dVector(queries))
print(output[:10])

[True, True, True, True, True, True, True, True, True, True]


## Voxel Carving

> The methods `create_from_point_cloud` and `create_from_triangle_mesh` create occupied voxels only on the surface of the geometry. It is however possible to carve a voxel grid from a number of depth maps or silhouettes. Open3D provides the methods `carve_depth_map` and `carve_silhouette` for voxel carving.
>
> The code below demonstrates the usage by first rendering depthmaps from a geometry and using those depthmaps to carve a dense voxel grid. The result is a filled voxel grid of the given shape.

:warning: **Since the file `sphere.ply` for the camera path was missing, I created a sphere witha desired resolution, nut the carving doesn't seem to work.**

In any case, this is the processing done:

- Mesh Preprocessing: Computes vertex normals for the mesh and preprocesses both the mesh and the camera sphere mesh to fit within a normalized cube.
- Setup Visualizer: Initializes an off-screen visualizer to render depth maps without displaying them.
- Carve Voxel Grid: Iteratively positions the camera at points on the camera_sphere, captures depth images from these viewpoints, and uses these depth images to carve the voxel grid. This means removing voxels that are not consistent with the observed depth maps.
- Depth Map Carving: If depth maps are used (use_depth=True), it carves the voxel grid using these depth images; otherwise, it uses silhouettes (binary visibility) for carving.
- Surface Extraction: Optionally, a surface can be extracted from the carved voxel grid using either a point cloud aggregation method or directly from the mesh, depending on the surface_method.
- Return Result: The function returns the combined voxel grid (carved and surface), the original voxel carving grid, and the surface voxel grid.

So in summary, voxelization is performed using voxel carving, i.e., a dense voxelizd block is carved using depth maps obtained from the mesh viewed from different camera positions.

In [31]:
def create_sphere_mesh(radius=1, resolution=5, display=False):
    # Create a sphere mesh with a given radius and resolution
    # resolution: Controls the density of vertices; increase for more vertices
    # resolution = 5 -> 42 vertices, 80 triangles
    # resolution = 6 -> 62 vertices, 120 triangles
    # ...
    # resolution = 10 -> 182 vertices, 360 triangles    
    sphere_mesh = o3d.geometry.TriangleMesh.create_sphere(radius=radius, resolution=resolution)
    sphere_mesh.compute_vertex_normals()  # Optional: Compute normals for better visualization
    if display:
        print("Sphere: Number of vertices:", len(sphere_mesh.vertices))
        print("Sphere: Number of triangles:", len(sphere_mesh.triangles))       
        o3d.visualization.draw_geometries([sphere_mesh], window_name="3D Sphere Visualization")
    return sphere_mesh


def xyz_spherical(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    r = np.sqrt(x * x + y * y + z * z)
    r_x = np.arccos(y / r)
    r_y = np.arctan2(z, x)
    return [r, r_x, r_y]


def get_rotation_matrix(r_x, r_y):
    rot_x = np.asarray([[1, 0, 0], [0, np.cos(r_x), -np.sin(r_x)],
                        [0, np.sin(r_x), np.cos(r_x)]])
    rot_y = np.asarray([[np.cos(r_y), 0, np.sin(r_y)], [0, 1, 0],
                        [-np.sin(r_y), 0, np.cos(r_y)]])
    return rot_y.dot(rot_x)


def get_extrinsic(xyz):
    rvec = xyz_spherical(xyz)
    r = get_rotation_matrix(rvec[1], rvec[2])
    t = np.asarray([0, 0, 2]).transpose()
    trans = np.eye(4)
    trans[:3, :3] = r
    trans[:3, 3] = t
    return trans


def preprocess(model):
    min_bound = model.get_min_bound()
    max_bound = model.get_max_bound()
    center = min_bound + (max_bound - min_bound) / 2.0
    scale = np.linalg.norm(max_bound - min_bound) / 2.0
    vertices = np.asarray(model.vertices)
    vertices -= center
    model.vertices = o3d.utility.Vector3dVector(vertices / scale)
    return model


def voxel_carving(mesh,
                  output_filename,
                  camera_space,
                  cubic_size,
                  voxel_resolution,
                  w=300,
                  h=300,
                  use_depth=True,
                  surface_method='pointcloud'):
    mesh.compute_vertex_normals()
    #camera_sphere = o3d.io.read_triangle_mesh(camera_path)
    camera_sphere = camera_space

    # setup dense voxel grid
    voxel_carving = o3d.geometry.VoxelGrid.create_dense(
        width=cubic_size,
        height=cubic_size,
        depth=cubic_size,
        voxel_size=cubic_size / voxel_resolution,
        origin=[-cubic_size / 2.0, -cubic_size / 2.0, -cubic_size / 2.0],
        color=[1.0, 0.7, 0.0])

    # rescale geometry
    camera_sphere = preprocess(camera_sphere)
    mesh = preprocess(mesh)

    # setup visualizer to render depthmaps
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=w, height=h, visible=False)
    vis.add_geometry(mesh)
    vis.get_render_option().mesh_show_back_face = True
    ctr = vis.get_view_control()
    param = ctr.convert_to_pinhole_camera_parameters()

    # carve voxel grid
    pcd_agg = o3d.geometry.PointCloud()
    centers_pts = np.zeros((len(camera_sphere.vertices), 3))
    for cid, xyz in enumerate(camera_sphere.vertices):
        # get new camera pose
        trans = get_extrinsic(xyz)
        param.extrinsic = trans
        c = np.linalg.inv(trans).dot(np.asarray([0, 0, 0, 1]).transpose())
        centers_pts[cid, :] = c[:3]
        ctr.convert_from_pinhole_camera_parameters(param)

        # capture depth image and make a point cloud
        vis.poll_events()
        vis.update_renderer()
        depth = vis.capture_depth_float_buffer(False)
        pcd_agg += o3d.geometry.PointCloud.create_from_depth_image(
            o3d.geometry.Image(depth),
            param.intrinsic,
            param.extrinsic,
            depth_scale=1)

        # depth map carving method
        if use_depth:
            voxel_carving.carve_depth_map(o3d.geometry.Image(depth), param)
        else:
            voxel_carving.carve_silhouette(o3d.geometry.Image(depth), param)
        print("Carve view %03d/%03d" % (cid + 1, len(camera_sphere.vertices)))
    vis.destroy_window()

    # add voxel grid survace
    print('Surface voxel grid from %s' % surface_method)
    if surface_method == 'pointcloud':
        voxel_surface = o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
            pcd_agg,
            voxel_size=cubic_size / voxel_resolution,
            min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
            max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2))
    elif surface_method == 'mesh':
        voxel_surface = o3d.geometry.VoxelGrid.create_from_triangle_mesh_within_bounds(
            mesh,
            voxel_size=cubic_size / voxel_resolution,
            min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
            max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2))
    else:
        raise Exception('invalid surface method')
    voxel_carving_surface = voxel_surface + voxel_carving

    return voxel_carving_surface, voxel_carving, voxel_surface

In [32]:
mesh = o3dex.get_armadillo_mesh()

output_filename = os.path.abspath("../models/voxelized.ply")
#camera_path = os.path.abspath("../models/sphere.ply")
camera_space = create_sphere_mesh(radius=1, resolution=10)
visualization = True
cubic_size = 2.0
voxel_resolution = 128.0

voxel_grid, voxel_carving, voxel_surface = voxel_carving(
    mesh, output_filename, camera_space, cubic_size, voxel_resolution)

[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 001/182
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 002/182
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 003/182
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 004/182
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 005/182
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 006/182
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.
Carve view 007/182
[Open3D WARNING] [Vi

In [33]:
print("surface voxels")
print(voxel_surface)
o3d.visualization.draw_geometries([voxel_surface])

print("carved voxels")
print(voxel_carving)
o3d.visualization.draw_geometries([voxel_carving])

print("combined voxels (carved + surface)")
print(voxel_grid)
o3d.visualization.draw_geometries([voxel_grid])

surface voxels
VoxelGrid with 386687 voxels.
carved voxels
VoxelGrid with 17282 voxels.
combined voxels (carved + surface)
VoxelGrid with 403969 voxels.
